In [2]:
# import Glop linear solver package
from ortools.linear_solver import pywraplp as glp

In [3]:
# input parameters
Investments = ['Unidyne Corp.', 'Col. Mustard Restaurant', 'First General REIT', 'Metropolitan Electric',
               'Western Union Corp.', 'Lemonville Transit', 'Fairview Apartment Partnership']
EAR = [0.150, 0.170, 0.175, 0.118, 0.122, 0.120, 0.220] # list of Expected Annual Return of investments
LF = [100, 100, 100, 95, 92, 79, 0] # list of Liquidity Factors of investments
RF = [60, 70, 75, 20, 30, 22, 50] # list of Risk Factors of investments
maxInvest = [80000, 80000, 80000, 80000, 80000, 80000, 80000] # max amount allowed to invest
minAverage_LiqFac = 65 # minimum weighted average liquidity factor
maxAverage_RisFac = 55 # maximum weighted average risk factor
minDebtInvestment = 90000 # minimum investment required in Debt category
cash = 400000

In [4]:
# initialize LP model object
mymodel = glp.Solver('Optimal Portfolio', glp.Solver.GLOP_LINEAR_PROGRAMMING)

In [5]:
# create decision variables
invest = list(range(len(Investments)))
for i in range(len(Investments)):
    invest[i] = mymodel.NumVar(0, mymodel.infinity(), Investments[i])
    # invest[i] = $ invested in investment i; where i = 0(Uni Eq.), 1(Col. Must.), 2(1st Gen REIT), 3(Met. Elec.), 
    # 4(Western), 5(Lem. Trans.), 6(Fair. Apt.)   

In [6]:
# define objective function
TotReturn = mymodel.Objective() # objective function: total expected return after tax
TotReturn.SetMaximization()
for i in range(len(Investments)):
    TotReturn.SetCoefficient(invest[i], EAR[i])

In [7]:
# set budget constraint
budget_constr = mymodel.Constraint (0, cash, 'budget')
for i in range(len(Investments)):
    budget_constr.SetCoefficient(invest[i], 1)

In [8]:
# set maximum investments constraints
maxInvest_constr = list(range(len(Investments)))
for i in range(len(Investments)):
    maxInvest_constr[i] = mymodel.Constraint(0, maxInvest[i], 'max of ' + Investments[i])
    maxInvest_constr[i].SetCoefficient(invest[i], 1)

In [9]:
# set weighted liquidity factor constraint
liquidity_constr = mymodel.Constraint(minAverage_LiqFac, mymodel.infinity(), 'liquidity')
for i in range(len(Investments)):
    liquidity_constr.SetCoefficient(invest[i], LF[i]/cash)

In [10]:
# set weighted risk factor constraint
risk_constr = mymodel.Constraint(0, maxAverage_RisFac, 'risk')
for i in range(len(Investments)):
    risk_constr.SetCoefficient(invest[i], RF[i]/cash)

In [11]:
# set minimum investment for debt
debt_constr = mymodel.Constraint(minDebtInvestment, mymodel.infinity(), 'debt')
for i in range(3):
    debt_constr.SetCoefficient(invest[i+3], 1)

In [12]:
# Solve the model and print optimal solution
status = mymodel.Solve()                 # solve mymodel and display the solution

print('Solution Status =', status)
print('Number of variables =', mymodel.NumVariables())
print('Number of constraints =', mymodel.NumConstraints())

print('Optimal Solution:')

# The objective value of the solution.
print('Total Return = %.2f' % TotReturn.Value())

Solution Status = 0
Number of variables = 7
Number of constraints = 11
Optimal Solution:
Total Return = 66555.00


In [13]:
# Display the investments
for i in range(len(Investments)):
    print('%30s \t %.2f' % (Investments[i], invest[i].solution_value()))
    

                 Unidyne Corp. 	 70000.00
       Col. Mustard Restaurant 	 80000.00
            First General REIT 	 80000.00
         Metropolitan Electric 	 0.00
           Western Union Corp. 	 27500.00
            Lemonville Transit 	 62500.00
Fairview Apartment Partnership 	 80000.00
